<p style='text-align: center'><a href=https://www.biozentrum.uni-wuerzburg.de/cctb/research/supramolecular-and-cellular-simulations/>Supramolecular and Cellular Simulations</a> (Prof. Fischer)<br>Center for Computational and Theoretical Biology - CCTB<br>Faculty of Biology, University of Würzburg</p>

<p style='text-align: center'><br><br>We are looking forward to your comments and suggestions. Please send them to: <br><br></p>
    
 <p style='text-align: center'>   <a href=andreas.kuhn@uni.wuerzburg.de>andreas.kuhn@uni.wuerzburg.de</a> or <a href=sabine.fischer@uni.wuerzburg.de>sabine.fischer@uni.wuerzburg.de</a></p>

<h1><p style='text-align: center'> Introduction to Julia </p></h1>


## File management

This notebook gives you a short introduction how to  manage paths, create/delete folders and to import/export data with Julia.   

### 1. Paths or where are we? 

A path is the address of a file or folder on a harddrive of your device. Paths can either be given in absolute terms or relative towards other paths. The function `pwd()` returns the absolute path of this notebook/the current Julia session. 

In [1]:
path_notebook = pwd()

"C:\\Users\\Andreas\\Dropbox\\phd\\Courses\\WS2022_Julia\\Part_7_File_Management"

Relative paths are always given in relation to the absolute path of the current notebook/Julia session (the return value of `pwd()`). It is possible to change this path, but we strongly advice you not to and always keep `pwd()` at its default value. 

If you want to get the contents of a folder you can use the  `readdir()` function with a path as argument.  `readdir()` returns the relative paths of all contents of a folder as a Vector of `Strings`. 

In [3]:
readdir(path_notebook)  # absolute path

23-element Vector{String}:
 ".ipynb_checkpoints"
 "AliceInWonderland.txt"
 "DrugScreen1.csv"
 "DrugScreen2.csv"
 "DrugScreen3.csv"
 "File_Management_solutions.ipynb"
 "Julia_File_Management.ipynb"
 "Untitled.ipynb"
 "Untitled1.ipynb"
 "Weights.csv"
 "a3.txt"
 "data"
 "hansi.txt"
 "list3D.csv"
 "matrix1"
 "matrix1.jls"
 "matrix3.txt"
 "matrix4.txt"
 "numbers.csv"
 "numbers.txt"
 "oh_shit.webp"
 "rockets.txt"
 "weights"

As relative paths are always given in relation to the notebook path, the relative path to this notebook is just nothing. 

In [4]:
readdir()              # relative path    

23-element Vector{String}:
 ".ipynb_checkpoints"
 "AliceInWonderland.txt"
 "DrugScreen1.csv"
 "DrugScreen2.csv"
 "DrugScreen3.csv"
 "File_Management_solutions.ipynb"
 "Julia_File_Management.ipynb"
 "Untitled.ipynb"
 "Untitled1.ipynb"
 "Weights.csv"
 "a3.txt"
 "data"
 "hansi.txt"
 "list3D.csv"
 "matrix1"
 "matrix1.jls"
 "matrix3.txt"
 "matrix4.txt"
 "numbers.csv"
 "numbers.txt"
 "oh_shit.webp"
 "rockets.txt"
 "weights"

As you can see, an absolute path is quite long, annoying to deal with and depends on your machine and operation system.  Therefore, if possible you should always work with relative paths, as they are smaller and easily portable.

If you want to create a folder you can use the `mkpath()` function which creates folder(s) at the given path. 

In [5]:
mkpath("test_folder1")
mkpath("test_folder2/sub_folder/data")

"test_folder2/sub_folder/data"

In [6]:
readdir()

25-element Vector{String}:
 ".ipynb_checkpoints"
 "AliceInWonderland.txt"
 "DrugScreen1.csv"
 "DrugScreen2.csv"
 "DrugScreen3.csv"
 "File_Management_solutions.ipynb"
 "Julia_File_Management.ipynb"
 "Untitled.ipynb"
 "Untitled1.ipynb"
 "Weights.csv"
 "a3.txt"
 "data"
 "hansi.txt"
 "list3D.csv"
 "matrix1"
 "matrix1.jls"
 "matrix3.txt"
 "matrix4.txt"
 "numbers.csv"
 "numbers.txt"
 "oh_shit.webp"
 "rockets.txt"
 "test_folder1"
 "test_folder2"
 "weights"

The return value of `readdir()` has changed because you have created the folders `test_folder1` and `test_folder2` inside our notebook folder. To see what's inside `test_folder` you can use its relative path as argument for `readdir()`.

In [7]:
readdir("test_folder1")

String[]

`test_folder1` is emtpy. 

In [8]:
readdir("test_folder2/")

1-element Vector{String}:
 "sub_folder"

In [9]:
readdir("test_folder2/sub_folder/")

1-element Vector{String}:
 "data"

Whereas `test_folder2` contains the folder `subfolder` which itself contains the folder `data`.

#### Note: The slash or backslash question
If you are using this notebook on Windows, `pwd()` has probably returned paths similar to this one: `"C:\\Users\\hanswurst\\sfw_stuff\\Julia_course\\Part_7_File_Management"` with `\\` instead of `/` . Therefore, you could use double backslashes instead of slashes in your code as well. On Windows this would work perfectly fine, but as soon as you use the same notebook on Linux or Mac, everything would break. Therefore, we strongly advice you to always use slashes `/`  as they work everywhere. 



If you want to delete a file or an empty folder you can use the `rm()` function. 

In [10]:
rm("test_folder1")

If you want to delete a folder that is not empty you get an error message. 

In [11]:
rm("test_folder2")

LoadError: IOError: rm("test_folder2"): directory not empty (ENOTEMPTY)

To remove a folder together with all files/subfolders inside, you can add the keyword argument `recursive = true`.

Note: You should only use this if you exactly know what you are doing and where you are. Because, if you are in the wrong folder it can happen very easily that you delete half your harddrive in an instant.  



In [12]:
rm("test_folder2",recursive = true)

![title](oh_shit.webp)

In [13]:
readdir()

23-element Vector{String}:
 ".ipynb_checkpoints"
 "AliceInWonderland.txt"
 "DrugScreen1.csv"
 "DrugScreen2.csv"
 "DrugScreen3.csv"
 "File_Management_solutions.ipynb"
 "Julia_File_Management.ipynb"
 "Untitled.ipynb"
 "Untitled1.ipynb"
 "Weights.csv"
 "a3.txt"
 "data"
 "hansi.txt"
 "list3D.csv"
 "matrix1"
 "matrix1.jls"
 "matrix3.txt"
 "matrix4.txt"
 "numbers.csv"
 "numbers.txt"
 "oh_shit.webp"
 "rockets.txt"
 "weights"

Note: There are a lot more [functions regarding the filesystem in Julia](https://docs.julialang.org/en/v1/base/file/), which provide a functionabilty almost as powerful as the Linux command line.

### 2. Import/Export data

When exporting/importing data, there are essential two questions to ask: 

#### Should the data be human readable  ? 

#### Will the data be used outside of Julia with other languages/programms ? 


###  2.1 Serialization 

If the answer to both questions is `no`, Julia has a very useful package for this purpose in its standard libary called `Serialization`.

In [14]:
using Serialization

This package provides two functions `serialize()` and `deserialize()`. 

`serialize()` expects two arguments. The first argument is the name of the file (its future path), that you want to create and the second argument is the Julia object you want to save. You can save every possible Julia object this way. It does not matter if it is a `String` an `array` a `dictionary` or arbitarly more complicated nested objects like `array of arrays, figures ...`. `serialize()` creates a Julia binary file at the given path. It is convention to use the ending `.jls` for such files.   


In [17]:
matrix1 = rand(100,100)

100×100 Matrix{Float64}:
 0.906652    0.486342   0.3663     …  0.666252  0.670681   0.768096
 0.305836    0.122622   0.789427      0.16382   0.076369   0.246166
 0.248224    0.608864   0.635913      0.906236  0.628231   0.326497
 0.527769    0.267786   0.196965      0.302819  0.754086   0.372124
 0.548042    0.632777   0.485998      0.647427  0.929124   0.592678
 0.00254681  0.506685   0.85951    …  0.79072   0.263636   0.434229
 0.874299    0.629236   0.8545        0.785434  0.0366204  0.969657
 0.24202     0.0412905  0.759161      0.853017  0.510505   0.442493
 0.440395    0.830371   0.811913      0.721099  0.826639   0.047809
 0.530148    0.157827   0.832705      0.50416   0.658007   0.267993
 0.598227    0.168188   0.135432   …  0.321893  0.367368   0.234831
 0.020105    0.787796   0.965         0.903477  0.04886    0.888485
 0.598985    0.42919    0.277002      0.553331  0.59476    0.122899
 ⋮                                 ⋱                       
 0.633263    0.165443   0.73071

In [18]:
serialize("matrix1.jls",matrix1)  
readdir()

23-element Vector{String}:
 ".ipynb_checkpoints"
 "AliceInWonderland.txt"
 "DrugScreen1.csv"
 "DrugScreen2.csv"
 "DrugScreen3.csv"
 "File_Management_solutions.ipynb"
 "Julia_File_Management.ipynb"
 "Untitled.ipynb"
 "Untitled1.ipynb"
 "Weights.csv"
 "a3.txt"
 "data"
 "hansi.txt"
 "list3D.csv"
 "matrix1"
 "matrix1.jls"
 "matrix3.txt"
 "matrix4.txt"
 "numbers.csv"
 "numbers.txt"
 "oh_shit.webp"
 "rockets.txt"
 "weights"

To import the created `jls` file, you can use `deserialize()`. This function takes only one argument, the path of the file you want to import and returns the restored Julia object. 

In [19]:
matrix1_restored = deserialize("matrix1.jls")

100×100 Matrix{Float64}:
 0.906652    0.486342   0.3663     …  0.666252  0.670681   0.768096
 0.305836    0.122622   0.789427      0.16382   0.076369   0.246166
 0.248224    0.608864   0.635913      0.906236  0.628231   0.326497
 0.527769    0.267786   0.196965      0.302819  0.754086   0.372124
 0.548042    0.632777   0.485998      0.647427  0.929124   0.592678
 0.00254681  0.506685   0.85951    …  0.79072   0.263636   0.434229
 0.874299    0.629236   0.8545        0.785434  0.0366204  0.969657
 0.24202     0.0412905  0.759161      0.853017  0.510505   0.442493
 0.440395    0.830371   0.811913      0.721099  0.826639   0.047809
 0.530148    0.157827   0.832705      0.50416   0.658007   0.267993
 0.598227    0.168188   0.135432   …  0.321893  0.367368   0.234831
 0.020105    0.787796   0.965         0.903477  0.04886    0.888485
 0.598985    0.42919    0.277002      0.553331  0.59476    0.122899
 ⋮                                 ⋱                       
 0.633263    0.165443   0.73071

`jls` files can only be read by Julia itself and cannot be opened by programms like text editor or excel. But this comes with the advantage of superior speed.  `Serialization` is around 10-1000 times faster than any other method to import/export data.  Therefore, if you work on your data in Julia exclusively and want to share it only with people that have/know Julia, `Serialization` is the way to go.

Comment: Serialization is mainly used in the internals of Julia to exchange objects between different processes/ Julia sessions, ... . Therefore, it is optimized for maximum performance but not necessarily for backwards compatibility. The devs have promised that it won't break between 1.xx versions of Julia, but if at some point a new x.0 version of Julia comes out, there might be issues importing old stored data into the new Julia version.
But as the past has shown, even packages, that are focused on long term storage, are not safe from [compatibilty issues](https://discourse.julialang.org/t/jld-jl-vs-jld2-jl/15287). Therefore, we believe that `Serialization` is the currently best and for sure fastest option to save huge junks of data in Julia.

###  2.2 Human readable data

Human readable means, that the exported data can be opened with other programms like text editor, excel, etc,... and then be read and interpreted by a human. The most general way to achive that, is to transform the data into `Strings` and then save them in a `.txt` file. There exist many different ways/standards to do this (`parse, JSON, JSON3, BSON,...`) one worse than the other.Therefore, the course will show you one Julia specific way to do that, because it is the least painful and most general. But exporting/importing your data as `Strings` is never fun/fast or desirable in any way and you should avoid it if possible. 

To transfrom any variable into a `String`, you can use a function that you already know:  `print()`

But this time, you have to combine it with another function `open()`, which opens/creates files. 

The `print()` function has an optional argument at the beginning, which specifies the target. The default value is the standard output/command line, which you have been using so far without noticing.  But in this case, you want to change it to said file. 

The `open()` function takes two arguments the first is the path of the file and the second is either `w` for write, `r` for read or `a` for append. 

In [22]:
print(open("matrix3.txt","w"),matrix1)

You can also use another syntax construct that is often used when providing functions as arguments to functions. The `do` construct: 

``` julia
function1(argument1,...) do name
    
    function2(name,argument2,...)
    
end
    
```

which is essentially the same as 

``` julia
function2(function1(argument1,...), argument2,...)
    
  
```

In [24]:
open("matrix4.txt","w") do file
    print(file,matrix1)
end

You can `print` different datatypes with this method into a file: 

In [25]:
rockets_dic = Dict("SpaceX" =>"Starship", "NASA" =>"SLS","ULA" => "AtlasV")
number_vec = collect(1:50)
hansi = "hansi geht in die hütten" 

open("hansi.txt","w") do file
    print(file,hansi)
end

open("rockets.txt","w") do file
    print(file,rockets_dic)
end

open("numbers.txt","w") do file
    print(file,number_vec)
end

Now five new `.txt` files should have been created inside the folder of this notebook. You can use `Notepad`, `JupyterLab` or many other programms to inspect their contents. 

If you want to import the printed data back into Julia, you can use the `read()` function together with `open()`.

In [26]:
open("hansi.txt","r") do file
    read(file,String)
end 

"hansi geht in die hütten"

But the more convenient way to do that is to use the function `readline()`, which  takes the path to a file as argument. 

In [27]:
hansi_restored = readline("hansi.txt")

"hansi geht in die hütten"

`readline()` always returns a `Strings`. If the original Julia object was a `String` in the frist place, everything works fine, but you will run into problems if the restored data wasn't originally a string.

In [28]:
number_vec_restored_string = readline("numbers.txt")

"[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]"

This oject which was an `Int64[]` array before,  is now a `String`. How can you transform it back to an array of type`Int64[]` ? One possible solution to this problem is the `parse()` function. 


In [29]:
parse(Int64,number_vec_restored_string[2])

1

In [30]:
parse(Int64,number_vec_restored_string[29:30])

10

In [31]:
parse(Int64,number_vec_restored_string)

LoadError: ArgumentError: invalid base 10 digit '[' in "[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]"

The problem is, that `parse()` only works for the parts of the string that soley consist of numbers. Therefore, parsing single numeric types like `Int64` or `Float64` can be done easily, but not composite data types like `array` or e.g. `dicts`, which also include other characters like `,` or `[`,... in their `String` representation. 

Now the "fun" part starts and you have to transform the string to sth. that is "parseable".

Here is one possible way to make the string parseable: 

1. Cut of the the brackets at the start and end 

2. Transform the `String` into a array of `Strings` with the `split()` function. Whereas the `,` is the symbol that indicates a "spliting point" for the `split()` function

3. Apply the broadcasted `parse.()` version of the `parse() `function with the `.` operator to every entry of the array of `Strings`    

In [32]:
numb_vec_restored1 = parse.(Int64, split(number_vec_restored_string[2:end-1],","))
println(numb_vec_restored1)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]


Overall this procedure is lengthy, annoying and needs to be changed for every datatype. For example, it wouldn't be possible to parse a 2D array in this way.  

This arises the obvious question: Isn't there a better way to do this ? 

In many programming languages, the best answer would be `JSON`. `JSON` is a standardized file format to parse complex objects to human-readable Strings representations. It was originally derived from JavaScript (JSON is short for JavaScript Object Notation) but is language independent. Julia also provides a [JSON-package](https://docs.juliahub.com/JSON/uf6oy/0.21.0/).  This can be very useful when you want to exchange  data between languages. But the downsides are a lot of overhead and slow execution speed. 

`Julia` offers a more natural alternative, which is part of one of the languages core features called [metaprogramming](https://docs.julialang.org/en/v1/manual/metaprogramming/). Which essentially means that "Julia represents its own code as a data structure of the language itself". Or, in more under stable terms, Julia itself can read and understand Julia source code. Surprisingly, this is a quite rare feature among programming languages, as most of the time the "read and understand source code" part is handled by a separate unity called the compiler/interpreter and not by the language itself. This opens the door to many very advanced programming patterns like live creation & execution of Julia source code during runtime,macros, ...

You won't use such advanced features here, except for one small part: Julia can evaluate every `String` not only as a `String` but also as source code.  

The `Meta.pase()` function transform a `String` into a `Julia` expression which can be evaluated with `eval()`. 

In [33]:
test_expression = "z = 100"
eval(Meta.parse(test_expression))
z

100

This an obviously very complicated way to create a variable `z` with content `100`. But, this feature also offers a nice alternative to parse a `String` if the string represents correct Julia source code. In this case the String `number_vec_restored_string` is correct source code for the definition of an array. Therefore it can be evaluated: 

In [34]:
number_vec_restored2 = eval(Meta.parse(number_vec_restored_string))
number_vec_restored2

50-element Vector{Int64}:
  1
  2
  3
  4
  5
  6
  7
  8
  9
 10
 11
 12
 13
  ⋮
 39
 40
 41
 42
 43
 44
 45
 46
 47
 48
 49
 50

With this syntax you can import everything you have printed to a file with `print`. 

In [35]:
rocket_string = readline("rockets.txt")

"Dict(\"ULA\" => \"AtlasV\", \"SpaceX\" => \"Starship\", \"NASA\" => \"SLS\")"

In [36]:
eval(Meta.parse(rocket_string))

Dict{String, String} with 3 entries:
  "ULA"    => "AtlasV"
  "SpaceX" => "Starship"
  "NASA"   => "SLS"

In [37]:
matrix_string = readline("matrix4.txt")

"[0.9066517490396916 0.48634155163783954 0.366300338752823 0.14640766041060893 0.9586353253783576 0.5597539584708467 0.12506175502119898 0.6097907974568008 0.3742905585538756 0.39652449986835026 0.1900775461341323 0.7061552771268929 0.22554406711945707 0.922361130506073" ⋯ 192268 bytes ⋯ ".4214907235750689 0.6859828328678925 0.19582852132481132 0.4011156346607949 0.36632241436514157 0.8693149969067953 0.13740827291132374 0.844571001364559 0.35408054297083225 0.49300372159346484 0.9923658300672143 0.9157232187066267 0.5411624956642348 0.9435227517238686]"

In [38]:
eval(Meta.parse(matrix_string))

100×100 Matrix{Float64}:
 0.906652    0.486342   0.3663     …  0.666252  0.670681   0.768096
 0.305836    0.122622   0.789427      0.16382   0.076369   0.246166
 0.248224    0.608864   0.635913      0.906236  0.628231   0.326497
 0.527769    0.267786   0.196965      0.302819  0.754086   0.372124
 0.548042    0.632777   0.485998      0.647427  0.929124   0.592678
 0.00254681  0.506685   0.85951    …  0.79072   0.263636   0.434229
 0.874299    0.629236   0.8545        0.785434  0.0366204  0.969657
 0.24202     0.0412905  0.759161      0.853017  0.510505   0.442493
 0.440395    0.830371   0.811913      0.721099  0.826639   0.047809
 0.530148    0.157827   0.832705      0.50416   0.658007   0.267993
 0.598227    0.168188   0.135432   …  0.321893  0.367368   0.234831
 0.020105    0.787796   0.965         0.903477  0.04886    0.888485
 0.598985    0.42919    0.277002      0.553331  0.59476    0.122899
 ⋮                                 ⋱                       
 0.633263    0.165443   0.73071

#### Final Note: Even though, you now know various ways to write/read data from .txt files in Julia you should never do any of that unless you are forced to by powers out of your control.  

###  2.3 Human readable data and usable -> DataFrames.jl and CSV.jl

But what is the right way to save data in a humanredable, exchangeable and usable way without needing to do all the tedious stuff in the previous chapter ?  
The solution are the packages `DataFrames` and `CSV` . 

In [40]:
using DataFrames, CSV

For scientific applications, numerical data is usally shared in the `.csv` (comma separated values) format. Therefore, if you want to share your data you should always provide it as `.csv` files. The `CSV` package provides an simple interface to import and export csv formated data into Julia. This comes with the limitation/benefit that you are forced to format your data as a `DataFrame` object from the `DataFrames` package. 

A `DataFrame` is essential a table, where each colum is a `Vector` of any type. The `DataFrames` package additionally provides a lot of data analysis feature, which you will learn in the next lesson. 

If you want to save a vector as a `.csv` file you need to convert into a `Dataframe` first. 

In [41]:
number_vec = collect(10:10:120)
Number_df = DataFrame(number = number_vec)

Row,number
,Int64
1,10
2,20
3,30
4,40
5,50
6,60
7,70
8,80
9,90


A column can be accessed by its name and a `.`.

In [42]:
Number_df.number

12-element Vector{Int64}:
  10
  20
  30
  40
  50
  60
  70
  80
  90
 100
 110
 120

Then we can use the `CSV.write()` function to save our dataframe on the given path. 

In [44]:
CSV.write("numbers.csv",Number_df)

"numbers.csv"

The function `CSV.File(path)` returns a table object that can be transformed back into a Dataframe with the `DataFrame()` function.

In [45]:
numbers_table = CSV.File("numbers.csv")

12-element CSV.File:
 CSV.Row: (number = 10,)
 CSV.Row: (number = 20,)
 CSV.Row: (number = 30,)
 CSV.Row: (number = 40,)
 CSV.Row: (number = 50,)
 CSV.Row: (number = 60,)
 CSV.Row: (number = 70,)
 CSV.Row: (number = 80,)
 CSV.Row: (number = 90,)
 CSV.Row: (number = 100,)
 CSV.Row: (number = 110,)
 CSV.Row: (number = 120,)

In [46]:
numbers_df = DataFrame(numbers_table)

Row,number
,Int64
1,10
2,20
3,30
4,40
5,50
6,60
7,70
8,80
9,90


Comment: This might be the first time, that you encounter an operator ( in this case `.`), that has different effects depending on the context. Once as a way to apply a function elementwise on a collection,  second as a way to access subobjects (`Vectors`) of a more complex object (`Dataframe`) and third to call specific functions from a pacakge (`CSV.write`). Don't worry, this might be confusing at first but is pretty common in programming. You get used to it very fast ! 

You can also use these two functions nested. The `CSV.File` function also has some useful additional keyword arguments like `header` which specifies which row should be read as the head of the table.

In [48]:
DataFrame(CSV.File("DrugScreen1.csv",header = 1)) 

Row,ObjectID,Label,Centroid,MeanIntensity,StandardDeviationIntensity,MaxIntensity,MinIntensity,TotalIntensity,Area,Count,EquivalentDiskRadius,Elongation,Eccentricity,Circularity,BoundingBox
,Int64,Int64,String,Float64,Float64,Float64,Float64,Float64,Float64,Int64,Float64,Float64,Float64,Float64,String31
1,5,5,{448.72869022869025. 497.8035343035343},0.0156262,0.000931721,0.0179599,0.0133364,7.5162,484.625,481,12.3736,0.106691,0.449443,0.93656,{{435.. 485.}. {463.. 509.}}
2,6,6,{583.3871165644172. 490.18957055214725},0.0118093,0.000430682,0.0129549,0.0106508,9.62455,819.375,815,16.1066,0.10129,0.438544,0.941424,{{566.. 474.}. {600.. 506.}}
3,7,7,{186.3943661971831. 478.2588028169014},0.0140465,0.000527996,0.0161288,0.0125429,7.97844,571.75,568,13.4462,0.111789,0.459435,0.977536,{{174.. 464.}. {200.. 492.}}
4,8,8,{267.6122047244094. 475.48228346456693},0.0144502,0.00083672,0.0168307,0.0125124,7.34072,512.25,508,12.7162,0.544601,0.890287,0.798317,{{248.. 459.}. {282.. 487.}}
5,9,9,{535.6371841155235. 472.5667870036101},0.0119992,0.000430933,0.0130617,0.0108492,6.64758,556.875,554,13.2794,0.283091,0.697166,0.921684,{{524.. 457.}. {547.. 488.}}
6,11,11,{83.59574468085107. 460.04255319148933},0.0137119,0.000525546,0.0151369,0.0122377,2.57784,189.0,188,7.73578,0.392051,0.793976,0.911685,{{75.. 454.}. {93.. 467.}}
7,12,12,{625.704938271605. 449.7320987654321},0.0127423,0.000449244,0.0137942,0.0113375,5.16062,407.625,405,11.3541,0.197412,0.596534,0.918163,{{615.. 437.}. {637.. 462.}}
8,13,13,{391.2029411764706. 437.6617647058823},0.013694,0.000698744,0.01561,0.0119936,4.65597,342.75,340,10.4031,0.338417,0.749872,0.894029,{{378.. 428.}. {404.. 446.}}
9,14,14,{312.9381338742393. 434.6926977687627},0.0135983,0.000792033,0.0155489,0.0115206,6.70397,496.375,493,12.527,0.378047,0.783055,0.892565,{{303.. 419.}. {323.. 451.}}


Note: If not all colums are displayed, you can change the number of vertical displayed characters in a jupyter notebook by the command: `ENV["COLUMNS"] = 250`. The default value is 100 which can be increased to a number that fits good to your screen. 

In [50]:
list_3d = DataFrame(CSV.File("list3D.csv"))

Row,Column1,0,1,2,3,4,5
,Int64,Int64,Int64,Int64,String,Int64,String
1,0,1,500,1,"[16.70828029922483, 13.868553757227511, 11.318761265247739]",18,"[(19.66748048024151, 13.70763285020259, 10.255750668200202), (17.241894987383603, 17.450619454634563, 11.982336240311415), (17.518626929335092, 10.662287015587191, 11.230259559034565), (20.185046423091915, 15.2264962674125, 14.033542356461785), (14.414073796460046, 11.874617928268126, 13.340524275922265), (15.504990729128854, 17.095866033685176, 14.600332348835215), (14.853137396184435, 10.618412896966946, 8.950511606720433), (15.5610547761724, 14.520326574317147, 15.765013811199585), (13.03876997982843, 11.091220636101303, 10.985933884998218), (18.36147867009802, 13.46595673647391, 8.40211455523821), (16.37178308725666, 16.762910942690212, 9.684552370298555), (19.045035031207416, 13.149444202557504, 15.101333199775258), (17.044257063947335, 11.951059506642586, 13.929228309967833), (20.65830972402022, 13.075508458916199, 13.16134618796188), (14.200326611569588, 14.53803916013269, 11.202833308136478), (14.57633673313276, 12.83378753547708, 8.647803630146877), (16.956533973045573, 11.369994633428695, 8.718758955194804), (19.252076927715986, 11.255904567197359, 12.435161957630692)]"
2,1,1,500,2,"[17.824286183314427, 20.263867851125152, 9.778457454229168]",15,"[(17.241894987383603, 17.450619454634563, 11.982336240311415), (18.72756046866427, 18.754550169380447, 12.910519566467242), (21.225373174193475, 17.10771428259479, 9.468255974942164), (16.206623935840966, 22.247507600460533, 10.584757929035629), (16.37178308725666, 16.762910942690212, 9.684552370298555), (16.125991094426148, 18.810519187938734, 9.521118298932219), (15.249718835119424, 19.891448799859614, 6.908360793407607), (20.12204229592246, 22.240528731683472, 8.876213978714226), (15.20885433614718, 19.410682347469873, 11.343217324835882), (17.55611004262072, 21.427680108771035, 7.091130848274567), (20.127575736594242, 22.345328465274207, 12.6143552712428), (22.6349085934094, 19.721330083619048, 8.8049203498855), (18.387447265886387, 19.198315174472988, 7.148808298828085), (20.085619067764537, 18.08132716597848, 7.271390375621), (17.498454654610498, 21.070590734674195, 12.341620201265254)]"
3,2,1,500,3,"[9.406143976123007, 11.226401417223862, 9.4282192927851]",17,"[(7.816140205285367, 9.255026338601022, 7.560844909516232), (10.513020951127135, 11.393119291716264, 5.0994865747947715), (8.158760105591512, 12.52122685675443, 12.141637603093452), (12.608172932890636, 9.949801431544708, 9.238803344051197), (10.04280874866686, 7.480804369434725, 9.614615781223012), (11.138583418648135, 11.900741124097722, 10.758676766588579), (7.9849671181794335, 13.779664701381083, 10.182475062529841), (7.69977616108985, 8.33067931653457, 9.521789606157684), (7.718616427559268, 10.150165909357563, 12.759693489317142), (7.89288692517149, 12.646720875592125, 7.246237363970902), (10.145817495236338, 10.250690821487797, 11.694428692239981), (9.179382434962168, 8.998289511404305, 6.021247124032736), (8.697013408378739, 8.381381019744618, 11.279871136000004), (6.653766597818299, 10.248593216973074, 9.063835650014765), (11.646216832119332, 12.80761984084087, 7.8676097424410845), (5.957112478019061, 12.002334626345652, 10.307107103611953), (9.890630813873338, 13.518656678560161, 9.297016268829527)]"
4,3,1,500,4,"[13.569180767089106, 8.782800719314455, 15.317873049490633]",15,"[(17.044866049414466, 9.972893608704279, 17.973472914135506), (11.315707985957507, 8.194893991523838, 13.57993241377889), (16.569810068391906, 9.986231945710102, 13.21230443953004), (14.650817381814633, 9.834485805693328, 18.767207328199458), (14.414073796460046, 11.874617928268126, 13.340524275922265), (14.07063013504656, 11.362428127000735, 16.33706227443417), (13.495942658776848, 8.434574945985837, 17.908653453636873), (15.995466832722634, 9.288209280912088, 11.281722822424095), (12.224282245320218, 12.381004103334293, 15.950341107713388), (13.292963516226116, 6.9511576313425065, 13

Normally, the datatype of column will be automatically determined by the `DataFrame` package. In the case of `list_3d`, this did not work properly as the datatype is no primitive julia datatype. Therefore, it was interpreted as a `String`. We can use the same Metaprogramming functions `eval(Meta.parse())` to cast the strings on valid Julia data types.   

In [51]:
# We have to use the broadcasted versions of the functions with . as we want to parse every single entry and not the whole column. 
list_3d."3" = eval.(Meta.parse.(list_3d."3"))
list_3d."5" = eval.(Meta.parse.(list_3d."5"))
list_3d

Row,Column1,0,1,2,3,4,5
,Int64,Int64,Int64,Int64,Array…,Int64,Array…
1,0,1,500,1,"[16.7083, 13.8686, 11.3188]",18,"[(19.6675, 13.7076, 10.2558), (17.2419, 17.4506, 11.9823), (17.5186, 10.6623, 11.2303), (20.185, 15.2265, 14.0335), (14.4141, 11.8746, 13.3405), (15.505, 17.0959, 14.6003), (14.8531, 10.6184, 8.95051), (15.5611, 14.5203, 15.765), (13.0388, 11.0912, 10.9859), (18.3615, 13.466, 8.40211), (16.3718, 16.7629, 9.68455), (19.045, 13.1494, 15.1013), (17.0443, 11.9511, 13.9292), (20.6583, 13.0755, 13.1613), (14.2003, 14.538, 11.2028), (14.5763, 12.8338, 8.6478), (16.9565, 11.37, 8.71876), (19.2521, 11.2559, 12.4352)]"
2,1,1,500,2,"[17.8243, 20.2639, 9.77846]",15,"[(17.2419, 17.4506, 11.9823), (18.7276, 18.7546, 12.9105), (21.2254, 17.1077, 9.46826), (16.2066, 22.2475, 10.5848), (16.3718, 16.7629, 9.68455), (16.126, 18.8105, 9.52112), (15.2497, 19.8914, 6.90836), (20.122, 22.2405, 8.87621), (15.2089, 19.4107, 11.3432), (17.5561, 21.4277, 7.09113), (20.1276, 22.3453, 12.6144), (22.6349, 19.7213, 8.80492), (18.3874, 19.1983, 7.14881), (20.0856, 18.0813, 7.27139), (17.4985, 21.0706, 12.3416)]"
3,2,1,500,3,"[9.40614, 11.2264, 9.42822]",17,"[(7.81614, 9.25503, 7.56084), (10.513, 11.3931, 5.09949), (8.15876, 12.5212, 12.1416), (12.6082, 9.9498, 9.2388), (10.0428, 7.4808, 9.61462), (11.1386, 11.9007, 10.7587), (7.98497, 13.7797, 10.1825), (7.69978, 8.33068, 9.52179), (7.71862, 10.1502, 12.7597), (7.89289, 12.6467, 7.24624), (10.1458, 10.2507, 11.6944), (9.17938, 8.99829, 6.02125), (8.69701, 8.38138, 11.2799), (6.65377, 10.2486, 9.06384), (11.6462, 12.8076, 7.86761), (5.95711, 12.0023, 10.3071), (9.89063, 13.5187, 9.29702)]"
4,3,1,500,4,"[13.5692, 8.7828, 15.3179]",15,"[(17.0449, 9.97289, 17.9735), (11.3157, 8.19489, 13.5799), (16.5698, 9.98623, 13.2123), (14.6508, 9.83449, 18.7672), (14.4141, 11.8746, 13.3405), (14.0706, 11.3624, 16.3371), (13.4959, 8.43457, 17.9087), (15.9955, 9.28821, 11.2817), (12.2243, 12.381, 15.9503), (13.293, 6.95116, 13.7266), (9.85215, 7.97316, 15.7633), (10.2247, 11.5558, 16.8026), (13.0388, 11.0912, 10.9859), (11.9922, 5.69629, 16.5576), (16.1556, 7.35468, 15.5616)]"
5,4,1,500,5,"[6.94144, 20.0205, 6.53004]",14,"[(10.5014, 19.5419, 3.28028), (8.23179, 16.1086, 4.51316), (3.93537, 18.4663, 6.85256), (8.53566, 19.4953, 10.4464), (9.65017, 17.7262, 7.67492), (8.82479, 21.9196, 5.39857), (7.86557, 23.3178, 9.38423), (7.05459, 17.3192, 7.61341), (10.3126, 19.2751, 6.49922), (8.29704, 21.2313, 9.12617), (5.62192, 19.0869, 9.88155), (6.87828, 16.8562, 2.80979), (4.98811, 19.7836, 7.94851), (8.63816, 17.8623, 1.91837)]"
6,5,1,500,6,"[9.45466, 6.14698, 2.75099]",13,"[(13.747, 10.0128, 0.991005), (11.9211, 14.5375, 0.352456), (6.24115, 6.41415, 5.08804), (10.7823, 2.04274, 6.65398), (11.5968, 7.86713, 3.98918), (10.0453, 6.23601, 5.04167), (9.75399, 8.6086, 3.22183), (5.83015, 8.81254, 3.78562), (7.19918, 11.8165, 2.0904), (11.267, 4.35868, 3.67304), (8.09001, 4.82782, 5.97559), (13.0302, 6.45331, 3.64867), (5.29258, 11.9669, 2.95101)]"
7,6,1,500,7,"[1.9912, 12.4126, 11.9962]",12,"[(5.95711, 12.0023, 10.3071), (4.42255, 10.6519, 12.7057), (1.24176, 13.7199, 10.657), (0.309762, 11.0693, 13.4908), (3.27143, 10.2888, 10.688), (1.94381, 12.0507, 8.75705), (3.3541, 9.87308, 14.6917), (1.11216, 9.93152, 11.9653), (5.22032, 12.7094, 12.6732), (1.02164, 15.277, 14.8458), (2.90362, 13.7699, 13.8908), (3.85668, 14.576, 10.2132)]"
8,7,1,500,8,"[14.8531, 10.6184, 8.95051]",13,"[(16.7083, 13.8686, 11.3188), (13.6569, 11.0488, 5.01754), (17.1397, 9.15679, 7.48044), (16.7126, 9.68253, 4.92135), (12.6082, 9.9498, 9.2388), (17.5186, 10.6623, 11.2303), (12.667, 7.78538, 6.63048), (15.9955, 9.28821, 11.2817), (13.171, 11.471, 7.74393), (13.0388, 11.0912, 10.9859), (15.47, 8.05377, 9.29133), (14.5763, 12.8338, 8.6478), (16.9565, 11.37, 8.71876)]"
9,8,1,500,9,"[17.2169, 1.69863, 14.8952]",9,"[(16.3109, 4.14948, 13.9808), (20.392, 2.5974, 16.4237), (19.1161, 1.88361, 13.9297), (16.0975, 2.43559, 16.8119), (14.2747, 1.79665, 12.7997), 

##  Short  Summary

- Julia can navigate through paths on your harddrive and create/load/delete folders and files
- Use `Serialization` to import and export data when only using Julia
- Printing/Reading Data into `.txt` files is painful and should be avoided if possible
- Use `DataFrames` and the `CSV` package to save and share your data in `.csv` files

## Exercises

All exercises in this course are divided into 3 different difficulty categories: <span style="color:green">easy</span>, <span style="color:orange">medium</span> and <span style="color:red">hard</span>. <span style="color:green">Easy</span> exercises should be solvable solely with the contents of the respective notebook. <span style="color:orange">Medium</span> often require the transfer of known concepts to new problems. Therefore, it might be necessary to look up some old notebooks or to use your creativity and curiosity to combine seemingly unrelated stuff. <span style="color:red">Hard</span> exercises take this concept one step further and might require you to use additional resources like the official documentation, google, StackOverflow,... . 

### <p style='color: green'>easy</p>


1. Get the number of files in the current working directory.


2. Count the number of .txt files in the current working directory.


3. Create a subfolder named "data" inside the folder of the notebook


4. Create and save a vector of Float64 as a `.jls` file inside data 


5. Load the `jls` file and delete it afterwards. 

### <p style='color: orange'>medium</p>


6. Create  20 vectors that each contain 30 ascending numbers like ` vec1 = [1,...,30] vec2 = [31,...,60],...`. Save each vector as seperate `jls` file into the data folder. 

Hint: You do not need to give each vector a name. 

7. Write a function called import_Vectors that imports all vectors from the datafolder and creates one big vector called HUGE that contains all the numbers in ascending order.   

8. Apply the sinus function to Huge and plot it as a scatter plot with CairoMakie. Save the plot in the data folder. 

9. Make sure that the import_Vector functions still works, even thought there is now a plot file as well in the data folder. 

10. Import the dataset drugScreen2 from the file DrugScreen3.csv. Check that the import worked correctly. If not, adjust the import parameters and modify the columns. 


### <p style='color: red'>hard</p>

11. Create a new folder called weights. Copy the file  Weights.csv to that folder, only using Julia. Import the dataset "Weights" (Weights.csv). Check that the import worked correctly. If not, adjust the import parameters. Plot a histogram of the weights and save it as a PDF in a new subfolder in weights called plots.

12. Import the file AliceInWonderland.txt and count the number of words in the file. 
